In [2]:
import numpy as np
import torch
import MAS_library as MASL #pylians
import redshift_space_library as RSL
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

##Per i grafici 3d indagare plotly
from kymatio.torch import HarmonicScattering3D
from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D

# Reads Quijote halo catalogues
# https://quijote-simulations.readthedocs.io/en/latest/halos.html
import readfof

I cataloghi di aloni sono elenchi degli aloni presi come punti massivi

Ognuno ha una propria massa e un numero di particelle di CDM da cui è formato

In [1]:
import os

In [ ]:
snapnum = 2
z_dict = {4:0.0, 3:0.5, 2:1.0, 1:2.0, 0:3.0}
redshift = z_dict[snapnum]
for root, dirs, files in os.walk("/home/fuffolo97/TESI/Try_02-HaloCatalogue/Halos"):
    for file in files:
        if file.endswith(".0"):
             print(os.path.join(root, file))

In [2]:
# input file
# need to select a folder and there must be
# a folder named '/groups_00*snapnum*'
# snapdir = '/home/fuffolo97/TESI/Try_02-HaloCatalogue/Halos/fiducial/0'
snapnum = 2

# determine the reìdshift of the catalogue
z_dict = {4:0.0, 3:0.5, 2:1.0, 1:2.0, 0:3.0}
redshift = z_dict[snapnum]

In [3]:
readfof.FoF_catalog?

Init signature:
readfof.FoF_catalog(
    basedir,
    snapnum,
    long_ids=False,
    swap=False,
    SFR=False,
    read_IDs=True,
    prefix='/groups_',
)
Docstring:      <no docstring>
File:           ~/anaconda3/envs/venv/lib/python3.10/site-packages/readfof.py
Type:           type
Subclasses:     


In [5]:
# read the halo catalogue
datas = readfof.FoF_catalog(snapdir, snapnum, read_IDs=False)

In [6]:
# how to get the propeties of the halos
pos_h = datas.GroupPos/1e3              # positions in Mpc/h
mass = datas.GroupMass * 1e10           # masses in M_sun/h
# vel_h = datas.GroupVel * (1.0*redshift) # peculiar velocities in km/s
# Npart = datas.GroupLen                  # number of CDM particles in the halo

In [15]:
np.shape(pos_h)

(195524, 3)

#### Create density matrix from Quijote Halo simulation

In [1]:
import sys

sys.path.insert(1, '/home/fuffolo97/TESI/MyFunc')
from myCIC import cic

In [9]:
dens = cic(pos_h, mass, 128, 1000)

time: 3m 7.1s

#### Calculate scattering transform coefficients

In [ ]:
# parameters for HarmonicScattering3D
M, N, O = 128, 128, 128
J, L = 4, 4

In [11]:
S = HarmonicScattering3D(J=J, L=L, shape=(M, N, O), sigma_0=0.8, integral_powers=[0.8])

In [12]:
# argument of "torch.from_numpy(...)" must be numpy.float32()
Sx = S.scattering(torch.from_numpy(dens))

In [14]:
first_order = []
second_order = []

for i in range(L+1):
    for j in range(J+1):
        first_order.append(Sx[j, i, 0])
    second_order.append(Sx[5:,i,0].numpy())